<a href="https://colab.research.google.com/github/HackerJacky/TimeSeriesColab/blob/main/TS_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
from scipy.stats import skew

In [6]:
#1
sp500=yf.download('^gspc',start='2021-01-01',end='2025-04-22')
TW_0050=yf.download('0050.TW',start='2021-01-01',end='2025-04-22')
btc=yf.download('btc-usd',start='2021-01-01',end='2025-04-22')
gold=yf.download('gc=f',start='2021-01-01',end='2025-04-22')
bonds=yf.download('0050.TW',start='2021-01-01',end='2025-04-22')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [41]:
#2
data_sp=sp500['Close']
data_tw=TW_0050['Close']
data_btc=btc['Close']
data_gold=gold['Close']
data_bonds=bonds['Close']
ret_sp=data_sp.pct_change().dropna()
ret_tw=data_tw.pct_change().dropna()
ret_btc=data_btc.pct_change().dropna()
ret_gold=data_gold.pct_change().dropna()
ret_bonds=data_bonds.pct_change().dropna()
#平均數
print("S&P500",ret_sp.mean())
print("台灣0050ETF",ret_tw.mean())
print("比特幣",ret_btc.mean())
print("黃金",ret_gold.mean())
print("債券",ret_bonds.mean())

S&P500 Ticker
^GSPC    0.00037
dtype: float64
台灣0050ETF Ticker
0050.TW    0.00045
dtype: float64
比特幣 Ticker
BTC-USD    0.001209
dtype: float64
黃金 Ticker
GC=F    0.000564
dtype: float64
債券 Ticker
0050.TW    0.00045
dtype: float64


In [42]:
#2
##標準差
print("S&P500",ret_sp.std())
print("台灣0050ETF",ret_tw.std())
print("比特幣",ret_btc.std())
print("黃金",ret_gold.std())
print("債券",ret_bonds.std())

S&P500 Ticker
^GSPC    0.011096
dtype: float64
台灣0050ETF Ticker
0050.TW    0.013681
dtype: float64
比特幣 Ticker
BTC-USD    0.032098
dtype: float64
黃金 Ticker
GC=F    0.009413
dtype: float64
債券 Ticker
0050.TW    0.013681
dtype: float64


In [45]:
#2
##偏態
print(skew(ret_tw, bias=False))
print(skew(ret_sp, bias=False))
print(skew(ret_btc, bias=False))
print(skew(ret_gold, bias=False))
print(skew(ret_bonds, bias=False))

[-0.24941512]
[0.15849829]
[0.12883259]
[-0.30308367]
[-0.24941512]


In [16]:
#3 ADF test
res_SP = ts.adfuller(ret_sp,1)
res_TW = ts.adfuller(ret_tw,1)
res_BTC = ts.adfuller(ret_btc,1)
res_GOLD = ts.adfuller(ret_gold,1)
res_BONDS = ts.adfuller(ret_bonds,1)

In [36]:
#4
##S&P500
reg_sp_1=AutoReg(ret_sp,lags=1).fit() #AR(1)
reg_sp_2=AutoReg(ret_sp,lags=2).fit() #AR(2)
reg_sp_3=AutoReg(ret_sp,lags=3).fit() #AR(3)
arma_ret_sp = ARIMA(ret_sp, order=(1, 0, 1)).fit()

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [37]:
#4
##台灣0050
reg_tw_1=AutoReg(ret_tw,lags=1).fit()
reg_tw_2=AutoReg(ret_tw,lags=2).fit()
reg_tw_3=AutoReg(ret_tw,lags=3).fit()
arma_ret_tw=ARIMA(ret_tw,order=(1,0,1)).fit()

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [38]:
#4
##比特幣
reg_btc_1=AutoReg(ret_btc,lags=1).fit()
reg_btc_2=AutoReg(ret_btc,lags=2).fit()
reg_btc_3=AutoReg(ret_btc,lags=3).fit()
arma_ret_btc=ARIMA(ret_btc,order=(1,0,1)).fit()


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dat

In [39]:
#4
##黃金
reg_gold_1=AutoReg(ret_gold,lags=1).fit()
reg_gold_2=AutoReg(ret_gold,lags=2).fit()
reg_gold_3=AutoReg(ret_gold,lags=3).fit()
arma_ret_gold=ARIMA(ret_gold,order=(1,0,1)).fit()

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [40]:
#4
##債券
reg_bonds_1=AutoReg(ret_bonds,lags=1).fit()
reg_bonds_2=AutoReg(ret_bonds,lags=2).fit()
reg_bonds_3=AutoReg(ret_bonds,lags=3).fit()
arma_ret_bonds=ARIMA(ret_bonds,order=(1,0,1)).fit()

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
